In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # Add project root to Python path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from src.utils import load_train_data, load_test_data, resumetable, col_value_counts
train_df = load_train_data()
test_df = load_test_data()

In [3]:
all_data = pd.concat([train_df, test_df], ignore_index=True)
target = all_data['target']
all_data = all_data.drop(columns='target')

In [4]:
all_data = all_data.drop(columns=['ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin', 'ps_ind_18_bin', 'ps_ind_14'])

calc_features = [col for col in all_data.columns if 'calc' in col]
all_data = all_data.drop(columns=calc_features)

In [5]:
all_data_df = resumetable(all_data)
missing_data = all_data_df[all_data_df['Missing'] > 0]
missing_data.sort_values(by='Missing', ascending=False)
missing_data_list = missing_data.index.tolist()

Data shape: (1488028, 33)


In [6]:
# 결측값 지시자 생성 -> 원래 결측값이었다는 사실 모델에게 전달
for feature in missing_data_list:
    new_feature_name = f'{feature}_ismissing'
    all_data[new_feature_name] = (all_data[feature] == -1).astype(int)

In [7]:
# 중앙값 대체 
continuos_features_to_impute = ['ps_reg_03', 'ps_car_14', 'ps_car_12']

for feature in continuos_features_to_impute:
    all_data[feature] = all_data[feature].replace(-1, np.nan)
    median_val = all_data[feature].median()
    all_data[feature] = all_data[feature].fillna(median_val)

In [8]:
all_data['ps_car_11'] = all_data['ps_car_11'].replace(-1, 3)

In [9]:
# all_features = all_data.columns
# all_data = all_data.drop(columns=['ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin', 'ps_ind_18_bin', 'ps_ind_14'])
# for feature in all_features:
#     if ('calc' in feature):
#         all_data = all_data.drop(columns=[feature])